In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as func


#创建SparkSession。
spark = SparkSession.builder.getOrCreate()

#读取文件并存储到DataFrame中。
df = spark.read.csv('../Datasets/news/news_sentiment.csv', header=False)

#指定标签列，并对文本特征列的数据进行分词处理。
df = df.select(df._c0.alias('label'), func.split(df._c1, ' ').alias('words'))

#分割出训练和测试集。
(train_df, test_df) = df.randomSplit([0.8, 0.2], seed=2021)

21/10/22 11:07:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/22 11:08:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/22 11:08:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/10/22 11:08:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
from pyspark.ml.feature import CountVectorizer, StringIndexer, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline


#对标签数据进行数字化编码。
labelIndexer = StringIndexer(inputCol="label", outputCol="idx_label")

#对文本数据进行词频特征抽取。
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=500)

#对词频特征进行标准化转换。
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

#使用随机森林分类器。
classifier = RandomForestClassifier(labelCol="idx_label", featuresCol="scaled_features")

#使用Pipeline，构建标签编码、特征抽取、特征转换，以及模型分类的执行流程。
pipeline = Pipeline(stages=[labelIndexer, cv, scaler, classifier])

model = pipeline.fit(train_df)

predictions = model.transform(test_df)

In [3]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


evaluator = MulticlassClassificationEvaluator(labelCol="idx_label", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

#评估分类器的准确率。
print ('Spark-ML的随机森林分类器在news_sentiment测试集上的准确率为：%.2f%%。' %(accuracy * 100))

Spark-ML的随机森林分类器在news_sentiment测试集上的准确率为：66.04%。
